In [2]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup as BS
from konlpy.tag import Mecab
import pandas as pd

In [3]:

mecab = Mecab()

In [4]:
today = datetime.today().strftime("%Y%m%d")

In [5]:
base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=101&date=" + today

In [8]:
def get_news_links(page):
    url = base_url + "&page=" + str(page)
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    links = []
    for a in soup.select("ul.type06_headline li dl dt a"):
        links.append(a["href"])
    for a in soup.select("ul.type06 li dl dt a"):
        links.append(a["href"])
    
    return links

def get_news_content(url):
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    title_tag = soup.select_one("h2.media_end_head_headline")
    content_tag = soup.find('article',{'id':'dic_area'})
    
    if title_tag and content_tag:
        title = title_tag.get_text().strip()
        content = content_tag.get_text().strip()
        return title, content
    else:
        return None, None

In [9]:
# 크롤링 시작
news_links = []
page = 1

while True:
    links = get_news_links(page)
    if not links or any(link in news_links for link in links):
        break
    news_links.extend(links)
    page += 1

In [12]:
# 뉴스 기사 내용 크롤링
news_contents = []
for link in news_links[:100]:
    try:
        title, content = get_news_content(link)
        news_contents.append((title, content))
    except Exception as e:
        print(f"Failed to get content from {link}: {e}")

In [13]:
# Create a DataFrame to store titles and nouns
df = pd.DataFrame(columns=["Title", "Nouns"])

# Text analysis using Mecab and store in DataFrame
for i, (title, content) in enumerate(news_contents, start=1):
    if content:
        nouns = mecab.nouns(content)
        new_row = pd.DataFrame({"Title": [title], "Nouns": [nouns]})
        df = pd.concat([df, new_row], ignore_index=True)        

# Display the DataFrame
print(df)

                                             Title  \
0                                 [포토] 도예명장 작품 멋지네   
1                                 [포토] 도예명장 작품 멋지네   
2        우리금융 "동양생명· ABL생명도 M&A 검토중이나 아직 결정된 바 없어"   
3        우리금융 "동양생명· ABL생명도 M&A 검토중이나 아직 결정된 바 없어"   
4                       미국 5월 신규주택판매 4월보다 11.3% 감소   
5                       미국 5월 신규주택판매 4월보다 11.3% 감소   
6                                매일유업, ‘피크닉 제로’ 출시   
7                                매일유업, ‘피크닉 제로’ 출시   
8               한일 산업장관, '수소공급망 개발 워킹그룹' 신설 합의(종합)   
9               한일 산업장관, '수소공급망 개발 워킹그룹' 신설 합의(종합)   
10                 [사진] 제철 감자 반값 … ‘맥 감자튀김’도 판매 재개   
11                 [사진] 제철 감자 반값 … ‘맥 감자튀김’도 판매 재개   
12                 한은 강릉본부 강원영동 경제포럼 '부동산시장 동향' 분석   
13                           강원 출생↓사망↑ 인구 데드크로스 심화   
14                  위기의 한국 사장님들… 자영업자 대출 연체율 3배 급등   
15                  위기의 한국 사장님들… 자영업자 대출 연체율 3배 급등   
16                  [포토]빅토르 앤 롤프 2024 오트쿠튀르 가을/겨울전   
17                  [포토]빅토르 

In [14]:
df

,Title,Nouns
0,[포토] 도예명장 작품 멋지네,"[관람객, 일, 서울, 강남구, 우수, 중소기업, 농, 특산품, 선물, 박람회, 전..."
1,[포토] 도예명장 작품 멋지네,"[관람객, 일, 서울, 강남구, 우수, 중소기업, 농, 특산품, 선물, 박람회, 전..."
2,"우리금융 ""동양생명· ABL생명도 M&A 검토중이나 아직 결정된 바 없어""","[우리, 금융, 대주주, 다자, 보험, 그룹, 구속, 양해, 각서, 체결, 디지털,..."
3,"우리금융 ""동양생명· ABL생명도 M&A 검토중이나 아직 결정된 바 없어""","[우리, 금융, 대주주, 다자, 보험, 그룹, 구속, 양해, 각서, 체결, 디지털,..."
4,미국 5월 신규주택판매 4월보다 11.3% 감소,"[전년, 동기, 급감, 모기지, 금리, 주택, 가격, 수요, 부진, 지속, 기사, ..."
5,미국 5월 신규주택판매 4월보다 11.3% 감소,"[전년, 동기, 급감, 모기지, 금리, 주택, 가격, 수요, 부진, 지속, 기사, ..."
6,"매일유업, ‘피크닉 제로’ 출시","[유업, 피크닉, 제로, 사진, 피크닉, 과일, 우유, 맛, 청량, 맛, 특징, 피..."
7,"매일유업, ‘피크닉 제로’ 출시","[유업, 피크닉, 제로, 사진, 피크닉, 과일, 우유, 맛, 청량, 맛, 특징, 피..."
8,"한일 산업장관, '수소공급망 개발 워킹그룹' 신설 합의(종합)","[美, 워싱턴, 회동, 청정, 수소, 암모니아, 민간, 협력, 장려, 핵심, 광물,..."
9,"한일 산업장관, '수소공급망 개발 워킹그룹' 신설 합의(종합)","[美, 워싱턴, 회동, 청정, 수소, 암모니아, 민간, 협력, 장려, 핵심, 광물,..."


In [11]:
len(news_links)

56